In [1]:
#!/usr/bin/env python

import os
import subprocess
import shutil
import binascii
import tempfile
import grass.script as gscript
import grass.script.setup as gsetup

# Configuration des chemins
gisbase = "/usr/lib/grass78"
gisdbase = tempfile.mkdtemp()  # Utiliser un répertoire temporaire pour la base de données GRASS
rep = '/home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/'

# INPUTS
input1 = os.path.join(rep, 'grille1_highres.tif')
input2 = os.path.join(rep, 'grille2_lowres.tif')
output = "fusion_grilles"

def reprojeter_raster(input_file, output_file):
    try:
        subprocess.run([
            'gdalwarp', 
            '-t_srs', 'EPSG:4326', 
            input_file, 
            output_file
        ], check=True)
        print(f"Raster reprojeté avec succès : {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de la reprojection : {e}")

# Reprojection des rasters
output_file_2 = input2[:-4]+'_reproj_WGS84.tif'
output_file_1 = input1[:-4]+'_reproj_WGS84.tif'
reprojeter_raster(input1, output_file_1)
reprojeter_raster(input2, output_file_2)

# Préparation des noms des rasters dans GRASS
raster1 = 'raster1_final'
raster2 = 'raster2_final'

# Créer une nouvelle location et mapset temporaires
location = binascii.hexlify(os.urandom(16)).decode()
mapset = 'PERMANENT'
location_path = os.path.join(gisdbase, location)
startcmd = f'grass78 -c epsg:4326 -e {location_path}'

print(f"Creating location with command: {startcmd}")
subprocess.run(startcmd, shell=True, check=True)

# Initialiser GRASS
gsetup.init(gisbase, gisdbase, location, mapset)

# Importer les rasters dans GRASS
gscript.run_command('r.in.gdal', input=output_file_1, output=raster1)
gscript.run_command('r.in.gdal', input=output_file_2, output=raster2)

# Définir la région d'étude
gscript.run_command('g.region', raster=f"{raster1},{raster2}")

# Exécuter la commande r.mblend
gscript.run_command('r.mblend',
                    high=raster1,
                    low=raster2,
                    output=output)

# Exporter le raster résultant
gscript.run_command('r.out.gdal', input=output, output=os.path.join(rep, output + '.tif'), format='GTiff')

# Nettoyer les fichiers temporaires
shutil.rmtree(location_path)
print(f"Temporary location {location_path} removed.")

print("Process completed successfully.")


Processing /home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/grille1_highres.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Raster reprojeté avec succès : /home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/grille1_highres_reproj_WGS84.tif
Processing /home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/grille2_lowres.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Raster reprojeté avec succès : /home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/grille2_lowres_reproj_WGS84.tif
Creating location with command: grass78 -c epsg:4326 -e /tmp/tmp7hcsq0um/8fd626ee4cfe40b571c75b685d948bf4


Démarrage du SIG GRASS ...
Création du nouveau Secteur GRASS <8fd626ee4cfe40b571c75b685d948bf4> ...
Cleaning up temporary files...
Cleaning up temporary files...
Importing raster map <raster1_final>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Importing raster map <raster2_final>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100


Region:
{'projection': 3, 'zone': 0, 'n': -16.995, 's': -24.605, 'w': 162.395, 'e': 169.305, 'nsres': 0.00481645569620253, 'ewres': 0.00481533101045298, 'rows': 1580, 'cols': 1435, 'cells': 2267300}


[r.mblend] Resampling low resolution raster to higher resolution
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
[r.mblend] Multiplying low resolution by zero
[r.mblend] Multiplying high resolution by zero
[r.mblend] Computing extent of low resolution
Extraction des entités surfaciques...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Écriture des lignes...
   0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68  72  76  80  84  88  92  96 100
Construction de la topologie pour la carte vectorielle
<tmp_983333@PERMANENT>...
Référencement des primitives ...
Construction des surfaces ...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100

CalledModuleError: Module run None r.out.gdal input=fusion_grilles output=/home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/fusion_grilles.tif format=GTiff ended with error
Process ended with non-zero return code 1. See errors in the (error) output.

In [3]:
os.path.join(rep, output + '.tif')

'/home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/fusion_grilles.tif'

In [5]:
gscript.run_command('r.out.gdal', input=output, output=os.path.join(rep, output + '_final.tif'), format='GTiff')

Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Float64>
Input raster map contains cells with NULL-value (no-data). The value -nan
will be used to represent no-data values in the input map. You can specify
a nodata value with the nodata option.
Exporting raster data to GTiff format...
ERROR 6: /home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/fusion_grilles_final.tif, band 1: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal terminé. File
</home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge/fusion_grilles_final.tif>
created.


0